In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
import multiprocessing as mp
from datetime import date,timedelta
import time as t
import matplotlib.pyplot as plt
import defs

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

%matplotlib inline
 
plt.rcParams['figure.figsize']=(20,10)
plt.style.use('ggplot')

In [ ]:
market_dfset = {}
dfset = {}
modelset = {}
futureset = {}
forecastset = {}
figureset = {}
legendset = {}
tickers = []

manager = mp.Manager()
tickers = manager.list()

#get file object
f = open("list", "r")

while(True):
    #read next line
    line = f.readline()
    #if line is empty, you are done with all lines in the file
    if not line:
        break
    #you can access the line
    tickers.append(line.strip())

#close file
f.close()

market_dfset=manager.dict()
modelset=manager.dict()
forecastset=manager.dict()

p = {}
for ticker in tickers:
    p[ticker]= mp.Process(target=defs.run_prophet_daily,args=(tickers,ticker,market_dfset,modelset,forecastset))
    
l = len(tickers)
c = mp.cpu_count()
for i in range(0, l, c):
    for j in range(0,c):
        if (i+j<l):
            p[tickers[i+j]].start()
    for j in range(0,c):
        if (i+j<l):
            p[tickers[i+j]].join()

In [ ]:
for ticker in tickers:
    print(ticker)
    figureset[ticker] = market_dfset[ticker]["Close"].plot()
    legendset[ticker]=figureset[ticker].legend() #get the legend
    legendset[ticker].get_texts()[0].set_text(ticker) #change the legend text
    plt.show()
    
for ticker in tickers:
    print(ticker)
    print(forecastset[ticker][['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(n=30))
    figure=modelset[ticker].plot(forecastset[ticker])  
    fig = modelset[ticker].plot(forecastset[ticker])
    a = add_changepoints_to_plot(fig.gca(), modelset[ticker], forecastset[ticker])
    figure2=modelset[ticker].plot_components(forecastset[ticker])
    plt.show()
    print('\n')

# Plotting the forecast

In [ ]:
two_yearset = {}

With the data that we have, it is hard to see how good/bad the forecast (blue line) is compared to the actual data (black dots).  Let's take a look at the last 800 data points (~2 years) of forecast vs actual without looking at the future forecast (because we are just interested in getting a visual of the error between actual vs forecast).

In [ ]:
for ticker in tickers:
    two_yearset[ticker] = forecastset[ticker].set_index('ds').join(market_dfset[ticker])
    two_yearset[ticker] = two_yearset[ticker][['Close', 'yhat', 'yhat_upper', 'yhat_lower' ]].dropna()
    two_yearset[ticker]['yhat']=np.exp(two_yearset[ticker].yhat)
    two_yearset[ticker]['yhat_upper']=np.exp(two_yearset[ticker].yhat_upper)
    two_yearset[ticker]['yhat_lower']=np.exp(two_yearset[ticker].yhat_lower)
    two_yearset[ticker].tail()
    figureset[ticker]=two_yearset[ticker][['Close', 'yhat']].plot()
    figureset[ticker].plot(two_yearset[ticker].yhat_upper, color='black',  linestyle=':', alpha=0.5)
    figureset[ticker].plot(two_yearset[ticker].yhat_lower, color='black',  linestyle=':', alpha=0.5)
    figureset[ticker].set_title('Actual (Orange) vs Forecasted Upper & Lower Confidence (Black)')
    figureset[ticker].set_ylabel('Price')
    figureset[ticker].set_xlabel('Date')
    legendset[ticker]=figureset[ticker].legend() #get the legend
    legendset[ticker].get_texts()[0].set_text(ticker) #change the legend text
    plt.show()

In [ ]:
two_years_AE_set = {}
for ticker in tickers:
    two_years_AE_set[ticker] = (two_yearset[ticker].yhat - two_yearset[ticker].Close)
    print(ticker)
    print(two_years_AE_set[ticker].describe())
    print("R2 score: ",r2_score(two_yearset[ticker].Close, two_yearset[ticker].yhat))
    print("MSE score: ",mean_squared_error(two_yearset[ticker].Close, two_yearset[ticker].yhat))
    print("MAE score: ",mean_absolute_error(two_yearset[ticker].Close, two_yearset[ticker].yhat))
    print('\n')

In [ ]:
full_dfset = {}
for ticker in tickers:
    full_dfset[ticker] = forecastset[ticker].set_index('ds').join(market_dfset[ticker])
    full_dfset[ticker]['yhat']=np.exp(full_dfset[ticker]['yhat'])

In [ ]:
for ticker in tickers:
    print(ticker)
    fig, ax1 = plt.subplots()
    n = -1
    ax1.plot(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):].Close)
    ax1.plot(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):].yhat, color='black', linestyle=':')
    ax1.fill_between(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):].index, np.exp(full_dfset[ticker].between_time('09:30','16:00')['yhat_upper'][full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):]), np.exp(full_dfset[ticker].between_time('09:30','16:00')['yhat_lower'][full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):]), alpha=0.5, color='darkgray')
    ax1.set_title('Actual (Orange) vs Forecasted (Black) with Confidence Bands')
    ax1.set_ylabel('Price')
    ax1.set_xlabel('Date')
    plt.show()

    fig2, ax2 = plt.subplots()
    n = -1
    ax2.plot(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D')+pd.DateOffset(1,'D')].Close)
    ax2.plot(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D')+pd.DateOffset(1,'D')].yhat, color='black', linestyle=':')
    ax2.fill_between(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D')+pd.DateOffset(1,'D')].index, np.exp(full_dfset[ticker].between_time('09:30','16:00')['yhat_upper'][full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D')+pd.DateOffset(1,'D')]), np.exp(full_dfset[ticker].between_time('09:30','16:00')['yhat_lower'][full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D')+pd.DateOffset(1,'D')]), alpha=0.5, color='darkgray')
    ax2.set_title('Actual (Orange) vs Forecasted (Black) with Confidence Bands')
    ax2.set_ylabel('Price')
    ax2.set_xlabel('Date')
    plt.show()

    fig3, ax3 = plt.subplots()
    n = 0
    ax3.plot(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):].Close)
    ax3.plot(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):].yhat, color='black', linestyle=':')
    ax3.fill_between(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):].index, np.exp(full_dfset[ticker].between_time('09:30','16:00')['yhat_upper'][full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):]), np.exp(full_dfset[ticker].between_time('09:30','16:00')['yhat_lower'][full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):]), alpha=0.5, color='darkgray')
    ax3.set_title('Actual (Orange) vs Forecasted (Black) with Confidence Bands')
    ax3.set_ylabel('Price')
    ax3.set_xlabel('Date')
    plt.show()

    n = -1
    today = full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):].Close
    print("Today: Highest", today.nlargest(1))
    print("Today: Lowest", today.nsmallest(1))
    n = 0
    tomorrow = pd.merge(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):]['yhat'], pd.merge(np.exp(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):]['yhat_upper']), np.exp(full_dfset[ticker].between_time('09:30','16:00')[full_dfset[ticker].last_valid_index()+pd.DateOffset(n, 'D'):]['yhat_lower']),on='ds'),on='ds')
    print("Tomorrow: Highest Upper", tomorrow['yhat_upper'].nlargest(1))
    print("Tomorrow: Highest", tomorrow['yhat'].nlargest(1))
    print("Tomorrow: Lowest", tomorrow['yhat'].nsmallest(1))
    print("Tomorrow: Lowest Lower", tomorrow['yhat_lower'].nsmallest(1))
    print('\n')
    pd.merge(today,tomorrow,left_index=True,right_index=True).to_csv(ticker+'-daily-1Days.csv')